# ANN hyper parameter search and analysis with tensorboard 

A.Geron Ch.10

## Question 1: 

#### Investigate the TensorBoard interface ( try to invoke it in the browser using url  localhost:6006 ) . What elements are available for review? How information is organized in tensorboard?

## Question 2:

#### Compare best model parameters and accuracy found by random search and hyperban search. Which accuracy is better? Compare both searches in TensorBoard. How both searches differ with reagrds to changes of val_accuracy across the training epochs? 

## Question 3:

#### If we wanted to access results of both hyperparameters searches in one TensorBoard  window - how would we achieve that?

#### Fashion MNIST images 

In [2]:
import tensorflow as tf
import keras as keras
import numpy as np

#### load the dataset

In [15]:
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]

X_train = X_train / 255
X_valid = X_valid / 255 
X_test = X_test / 255

In [9]:
X_train.shape

(55000, 28, 28)

In [10]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
"Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]
class_names[y_train[0]]

'Ankle boot'

#### Create model to work with

In [11]:
tf.random.set_seed(42)
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=[28, 28]))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300, activation="relu"))
model.add(tf.keras.layers.Dense(100, activation="relu"))
model.add(tf.keras.layers.Dense(10, activation="softmax"))

#### Also recall that we can pass list of layers

In [16]:
model = tf.keras.Sequential([
tf.keras.layers.Flatten(input_shape=[28, 28]),
tf.keras.layers.Dense(300, activation="relu"),
tf.keras.layers.Dense(100, activation="relu"),
tf.keras.layers.Dense(10, activation="softmax")
])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_6 (Flatten)         (None, 784)               0         
                                                                 
 dense_18 (Dense)            (None, 300)               235500    
                                                                 
 dense_19 (Dense)            (None, 100)               30100     
                                                                 
 dense_20 (Dense)            (None, 10)                1010      
                                                                 
Total params: 266610 (1.02 MB)
Trainable params: 266610 (1.02 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


#### Compile , train and evaluate

In [18]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=30, verbose=0, validation_data=(X_valid, y_valid))
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.3208 - accuracy: 0.8856


[0.3208242952823639, 0.8855999708175659]

#### Use the model to make predictions

In [19]:
X_new = X_test[:3]
y_proba = model.predict(X_new)
y_proba.round(2)

1/1 [==============================] - 0s 58ms/step


array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.98],
       [0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]],
      dtype=float32)

#### Which class?

In [20]:
import numpy as np
y_pred = y_proba.argmax(axis=-1)
print(y_pred)
np.array(class_names)[y_pred]

[9 2 1]


array(['Ankle boot', 'Pullover', 'Trouser'], dtype='<U11')

#### What are correct answers?

In [21]:
y_new = y_test[:3]
y_new

array([9, 2, 1], dtype=uint8)

#### Save and restore models

In [26]:
model.save("my_keras_model", save_format="tf")
model1 = tf.keras.models.load_model("my_keras_model")

y_pred1 = model1.predict(X_test[:3])
y_pred1.round(2)

INFO:tensorflow:Assets written to: my_keras_model/assets


INFO:tensorflow:Assets written to: my_keras_model/assets


1/1 [==============================] - 0s 40ms/step


array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.98],
       [0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]],
      dtype=float32)

#### Using callbacks and TensorBoard

the ModelCheckpoint callback saves checkpoints of your model at regular intervals during training, by default at the end of each epoch in fit() method.

#### Create a function to create a directory in which the callbacks will be saved

In [28]:
from pathlib import Path
from time import strftime

def get_run_logdir(root_logdir="my_logs"):
    return Path(root_logdir) / strftime("run_%Y_%m_%d_%H_%M_%S")

run_logdir = get_run_logdir()
# e.g., my_logs/run_2022_08_01_17_25_59

print(run_logdir)

my_logs/run_2024_01_17_20_10_00


#### Use the TensorBoard callbacks to profile the model training (verbose =0 here, wait for process to complete)

In [ ]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir, profile_batch=(100, 200))
history = model.fit(X_train, y_train, epochs=30, verbose=0, validation_data=(X_valid, y_valid), callbacks=[tensorboard_cb])

#### Invoke a tensor board when process has completed , the checkpoints written under the root_logdir "my_logs". 

In [32]:
# UNCOMMENT

#%load_ext tensorboard
#%tensorboard --logdir=./my_logs

## Fine tuning Neural Network Hyperparameters

Find optimal number of layers, the number of neurons and the type of activation function to use in each layer, the type of optimizer to use, its learning rate, the batch size, and more. What combination of hyperparameters is the best for your task?

In [33]:
# UNCOMMENT ana RUN
#%pip install -q -U keras-tuner

In [38]:
import keras_tuner as kt

#### This is function to build models with various combinations of the hyperparameters 

In [43]:
def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=0.01, max_value=0.02, sampling="log")
    optimizer = hp.Choice("optimizer", values=["sgd", "adam"])
    if optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    for _ in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
        
    model.add(tf.keras.layers.Dense(10, activation="softmax"))
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

#### Random search tuner from keras_tuner

In [60]:
# initialize random search tuner , the function that builds models is passed as argument
# In order to inspect result with tensor board pass the tensor board callbacks to the search method
random_search_tuner = kt.RandomSearch(build_model, objective="val_accuracy", max_trials=5,
                                      overwrite=False, directory="fashion_dataset", 
                                      project_name="random_search", seed=42)
random_search_tuner.search(X_train, y_train, epochs=10, verbose=1, validation_data=(X_valid, y_valid),
                          callbacks=[keras.callbacks.TensorBoard("./fashion_dataset/random_search_log")])

Trial 5 Complete [00h 01m 28s]
val_accuracy: 0.8723999857902527

Best val_accuracy So Far: 0.8744000196456909
Total elapsed time: 00h 08m 24s


In [50]:
### Too many warnings
#top3_models = random_search_tuner.get_best_models(num_models=3)
#best_model = top3_models[0]

In [51]:
top3_params = random_search_tuner.get_best_hyperparameters(num_trials=3)
top3_params[0].values # best hyperparameter values

{'n_hidden': 5,
 'n_neurons': 70,
 'learning_rate': 0.012378244763480603,
 'optimizer': 'sgd'}

In [52]:
best_trial = random_search_tuner.oracle.get_best_trials(num_trials=1)[0]
best_trial.summary()

Trial 4 summary
Hyperparameters:
n_hidden: 5
n_neurons: 70
learning_rate: 0.012378244763480603
optimizer: sgd
Score: 0.8781999945640564


In [53]:
best_trial.metrics.get_last_value("val_accuracy")

0.8781999945640564

#### Invoke TensorBoard to inspect random search profiling 

In [62]:
%load_ext tensorboard
%tensorboard --logdir=./fashion_dataset/random_search_log

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## Hyperband tuner from keras_tuner. This is directed search that adjusts parameter values based on improvement of objective function. 

#### Hyper parameters function

#### Initialize hyperband tuner

In [ ]:
# What part of search process controls parameter hyperband_iterations? 

hyperband_tuner = kt.Hyperband( MyClassificationHyperModel(), objective="val_accuracy", seed=42, 
                               max_epochs=10, factor=3, hyperband_iterations=2,
                               overwrite=False, directory="fashion_dataset", project_name="hyperband_search" )

hyperband_tuner.search(X_train, y_train, epochs=10, verbose=1, validation_data=(X_valid, y_valid),
                          callbacks=[keras.callbacks.TensorBoard("./fashion_dataset/hyperband_search_log"),  tf.keras.callbacks.EarlyStopping(patience=2)])

# Print best model parameters

In [73]:
#%reload_ext tensorboard
#%tensorboard --logdir="./fashion_dataset/hyperband_search_log"